# パーセプトロン
このNotebookで、算数だけで単純なニューラルネットワーク（パーセプトロン）を構築し、「AND」と「OR」の論理演算を学習させてみよう

まず、必要なパッケージ（機能）を挿入しよう

In [1]:
# numpyはベクトル、または行列を管理しやすいパッケージである。
import numpy as np 

## numpyの操作
まず、numpy配列の操作を少し把握しないといけない。numpyでは、「行」と「列」を別々で処理できる。例えば、この行列

\begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \end{bmatrix}

numpyでこうなる：

In [2]:
test_matrix = np.array([[1, 2, 3], [4, 5, 6]])

# 2行、3列であることを確認！
print(test_matrix.shape)

(2, 3)


In [3]:
#中身も確認
print(test_matrix)

[[1 2 3]
 [4 5 6]]


In [4]:
#また、1個の要素だけを見たい場合、0から始まる指数でアクセスできる：
print(test_matrix[1, 0])

4


In [5]:
# 1行目を確認
print(test_matrix[0,:])

[1 2 3]


In [6]:
# 2列目を確認
print(test_matrix[:,1])

[2 5]


In [7]:
# もちろん、行か列単位で計算もOK
print(test_matrix[0,:] + test_matrix[1,:])

[5 7 9]


In [8]:
# また、数値・演算・numpy も可能
# 例えば、足し算
print (1 + test_matrix)

[[2 3 4]
 [5 6 7]]


In [9]:
# 乗算
print (2 * test_matrix[0,:])

[2 4 6]


## パーセプトロンの構築
つぎ、入力２つ、出力１つがあるパーセプトロンを構築しよう。何回も作るかもしれないので、関数で実装：

In [10]:
def create_perceptron():    
    model = {
        # 荷重を -5 ~ 5 の乱数で正気化
        "weights" : np.random.uniform(-5, 5, 2),    
        # バイアスも！
        "bias"    : np.random.uniform(-5, 5, 1)}

    return model

モデルを１つ作って、中身を確認

In [11]:
model = create_perceptron()
print(model)

{'weights': array([-4.15468648,  0.66449053]), 'bias': array([4.25597411])}


当然ながら、乱数はみんなが違うので、統一するため、乱数の「シード」（種）を決めましょう

In [12]:
np.random.seed(12345)

model = create_perceptron()
print(model)

{'weights': array([ 4.29616093, -1.83624445]), 'bias': array([-3.16081188])}


## 推定
パーセプトロンの推定は:

$$y^{pred}_j = \sum^n_{i=1} x_{ij}w_i + bias$$

今回ば、入力は２つのみで、xは「入力配列」であり、1行あたりは、1サンプル（学習データの1点）にする

In [13]:
def predict(model, activation, x):
    
    # 各サンプルの入力とその荷重の乗算を足して、最後のバイアスをいれて…
    y = model["weights"][0] * x[:,0] + model["weights"][1] * x[:,1] + model["bias"]
    
    # 活性化を返す
    y = activation(y)
    return y

このタイミングで、活性化関数はあまり気にしないで、とりあえず何もしない「線形関数」でいきましょう

In [14]:
def linear_activation(x):
    return x

入力データを用意し、推定してみましょう！

In [15]:
x = np.array([
    [0, 0], 
    [0, 1], 
    [1, 0], 
    [1, 1]
], dtype = float)

print(x.shape)

(4, 2)


In [16]:
# 推定
y_pred = predict(model, linear_activation, x)
print(y_pred.shape)

(4,)


In [17]:
# 各入力データポイントの結果
print(y_pred)

[-3.16081188 -4.99705634  1.13534904 -0.70089541]


## 学習
当たり前が、まだ学習されていないので、推定はごみ。学習のため、損失関数、損失関数の勾配、そして荷重変化は以下の通りに（細かい説明はスライドで）

$$loss(W) = \frac{1}{2m}\sum_{j=1}^{m}(y^{true}_j-y^{pred}_j)^2$$
$$\nabla{loss(W)} = \Delta{W} = -\frac{1}{m}\sum_{j=1}^{m}(y^{true}_j-y^{pred}_j)\cdot X_j$$
$$W'=W + \Delta{W} \cdot lr$$


In [18]:
# 損失関数は'mean squared error'（平均二乗誤差）とする
def mean_squared_error(y_true, y_pred):
    error = y_true - y_pred
    mse = np.mean(error * error)
    return mse

In [19]:
# y_trueを定義しないと！ANDの場合は：
y_true = np.array([ 0, 0, 0, 1 ], dtype=float)

In [20]:
# さて、推定と実際のデータの損失は？
print(y_true)
print(y_pred)
print(mean_squared_error(y_true, y_pred))

[0. 0. 0. 1.]
[-3.16081188 -4.99705634  1.13534904 -0.70089541]
9.785841611861958


In [21]:
# 次、勾配関数を作ろう～
def mse_gradient(x, y_true, y_pred):
    
    error = y_true - y_pred

    grad_w = - x * error
    grad_b = - 1 * error

    return np.mean(grad_w, axis=0), np.mean(grad_b, axis=0)

In [22]:
# 勾配（荷重変化）はこんな感じ：
print (mse_gradient(x, y_true, y_pred))

ValueError: operands could not be broadcast together with shapes (4,2) (4,) 

あれ？なにこのエラー？

`ValueError: operands could not be broadcast together with shapes (4,2) (4,) `

なんか、shapeが合わないっぽい！

In [23]:
# shapeを確認すると：
error = y_true - y_pred
print(x.shape)
print(error.shape)

(4, 2)
(4,)


In [24]:
# なるほど。errorの「列」固定ではない（サイズ変えてもOKのShape）
# reshapeで固定しましょう
error = error.reshape(4, 1)
print(error.shape)

(4, 1)


In [25]:
#これならば, 乗算できるよね？
print(-x * error)

[[-0.         -0.        ]
 [-0.         -4.99705634]
 [ 1.13534904  0.        ]
 [-1.70089541 -1.70089541]]


In [26]:
# で、勾配関数を直そう～
def mse_gradient(x, y_true, y_pred):
    
    error = y_true - y_pred

    # errorのshapeがxのshapeが合わないため、reshapeが必要
    error = error.reshape(x.shape[0], 1)
    
    grad_w = - x * error
    grad_b = - 1 * error

    return np.mean(grad_w, axis=0), np.mean(grad_b, axis=0)

In [27]:
# 勾配（荷重変化）はこんな感じ：
print (mse_gradient(x, y_true, y_pred))

(array([-0.14138659, -1.67448794]), array([-2.18085365]))


それでは、必要な関数ができたので、学習関数を作ろう！

In [28]:
# 学習関数
def fit(model, activation, loss, gradient, learning_rate, x, y_true):
    
    # 推定
    y_pred = predict(model, activation, x)
    
    # 損失
    print("Loss:", loss(y_true, y_pred))
    
    # 勾配
    grad_w, grad_b = gradient(x, y_true, y_pred)
    
    # 荷重調整
    adjust_w = grad_w * learning_rate
    adjust_b = grad_b * learning_rate
    
    # モデル更新
    model["weights"] = model["weights"] - adjust_w
    model["bias"]    = model["bias"]    - adjust_b

In [29]:
# 試してみましょう！

print (y_true)

# before
y_pred = predict(model, linear_activation, x)
print (y_pred)

fit(model, linear_activation, mean_squared_error, mse_gradient, 0.01, x, y_true)

# after
y_pred = predict(model, linear_activation, x)
print (y_pred)

[0. 0. 0. 1.]
[-3.16081188 -4.99705634  1.13534904 -0.70089541]
Loss: 9.785841611861958
[-3.13900335 -4.95850292  1.15857145 -0.66092813]


In [30]:
# 同じことを何回も学習させる（epochs)
def fit_epochs(model, activation, loss, gradient, learning_rate, x, y_true, epochs):
    
    for i in range(epochs):
        fit(model, activation, loss, gradient, learning_rate, x, y_true)

In [31]:
fit_epochs(model, linear_activation, mean_squared_error, mse_gradient, 0.01, x, y_true, 3000)

Loss: 9.635265819401875
Loss: 9.488725362135831
Loss: 9.346096123854624
Loss: 9.20725789481097
Loss: 9.072094248326433
Loss: 8.940492421298066
Loss: 8.812343198481594
Loss: 8.687540800431716
Loss: 8.565982774983963
Loss: 8.447569892166161
Loss: 8.332206042431118
Loss: 8.21979813810552
Loss: 8.110256017953434
Loss: 8.00349235475594
Loss: 7.899422565811563
Loss: 7.79796472626521
Loss: 7.699039485176176
Loss: 7.602569984238688
Loss: 7.508481779071096
Loss: 7.416702762992597
Loss: 7.327163093208776
Loss: 7.23979511932992
Loss: 7.154533314148306
Loss: 7.071314206603104
Loss: 6.990076316863726
Loss: 6.910760093464696
Loss: 6.833307852427144
Loss: 6.757663718304205
Loss: 6.683773567089455
Loss: 6.611584970929524
Loss: 6.541047144583863
Loss: 6.472110893576448
Loss: 6.404728563985925
Loss: 6.338853993822423
Loss: 6.2744424659409
Loss: 6.211450662442422
Loss: 6.149836620516381
Loss: 6.089559689678096
Loss: 6.03058049035769
Loss: 5.972860873797535
Loss: 5.916363883216912
Loss: 5.861053716203832


Loss: 0.4123685522566386
Loss: 0.4109041501336697
Loss: 0.409446174119819
Loss: 0.4079945949055264
Loss: 0.4065493833188792
Loss: 0.40511051032495304
Loss: 0.4036779470251538
Loss: 0.40225166465656487
Loss: 0.4008316345912959
Loss: 0.3994178283358359
Loss: 0.39801021753040844
Loss: 0.3966087739483308
Loss: 0.3952134694953759
Loss: 0.39382427620913774
Loss: 0.39244116625839887
Loss: 0.3910641119425024
Loss: 0.3896930856907261
Loss: 0.3883280600616592
Loss: 0.38696900774258386
Loss: 0.3856159015488577
Loss: 0.3842687144233007
Loss: 0.3829274194355843
Loss: 0.3815919897816238
Loss: 0.38026239878297424
Loss: 0.37893861988622796
Loss: 0.377620626662416
Loss: 0.3763083928064124
Loss: 0.37500189213634116
Loss: 0.3737010985929862
Loss: 0.37240598623920357
Loss: 0.3711165292593378
Loss: 0.36983270195864004
Loss: 0.3685544787626893
Loss: 0.36728183421681665
Loss: 0.36601474298553194
Loss: 0.3647531798519537
Loss: 0.36349711971724175
Loss: 0.3622465376000318
Loss: 0.36100140863587404
Loss: 0.3597

Loss: 0.11032429251651679
Loss: 0.11014426445717888
Loss: 0.10996495102572193
Loss: 0.10978634923173539
Loss: 0.10960845609792863
Loss: 0.10943126866007107
Loss: 0.10925478396693258
Loss: 0.10907899908022417
Loss: 0.10890391107453917
Loss: 0.10872951703729433
Loss: 0.10855581406867156
Loss: 0.10838279928155954
Loss: 0.10821046980149611
Loss: 0.10803882276661032
Loss: 0.10786785532756546
Loss: 0.10769756464750159
Loss: 0.10752794790197903
Loss: 0.10735900227892169
Loss: 0.10719072497856073
Loss: 0.1070231132133787
Loss: 0.10685616420805367
Loss: 0.1066898751994037
Loss: 0.1065242434363318
Loss: 0.10635926617977068
Loss: 0.1061949407026283
Loss: 0.10603126428973324
Loss: 0.10586823423778054
Loss: 0.1057058478552778
Loss: 0.10554410246249138
Loss: 0.10538299539139309
Loss: 0.10522252398560682
Loss: 0.10506268560035577
Loss: 0.10490347760240967
Loss: 0.10474489737003227
Loss: 0.10458694229292928
Loss: 0.10442960977219623
Loss: 0.10427289722026688
Loss: 0.10411680206086171
Loss: 0.103961321

Loss: 0.06501162267656549
Loss: 0.06500329547383052
Loss: 0.06499499663560535
Loss: 0.06498672606170262
Loss: 0.06497848365230535
Loss: 0.0649702693079653
Loss: 0.06496208292960179
Loss: 0.06495392441849997
Loss: 0.06494579367630965
Loss: 0.06493769060504379
Loss: 0.06492961510707705
Loss: 0.0649215670851444
Loss: 0.06491354644233979
Loss: 0.06490555308211465
Loss: 0.06489758690827659
Loss: 0.06488964782498793
Loss: 0.06488173573676448
Loss: 0.06487385054847386
Loss: 0.06486599216533456
Loss: 0.06485816049291421
Loss: 0.06485035543712839
Loss: 0.0648425769042393
Loss: 0.06483482480085431
Loss: 0.06482709903392474
Loss: 0.06481939951074445
Loss: 0.06481172613894852
Loss: 0.06480407882651197
Loss: 0.06479645748174837
Loss: 0.06478886201330865
Loss: 0.06478129233017962
Loss: 0.06477374834168279
Loss: 0.06476622995747314
Loss: 0.06475873708753761
Loss: 0.06475126964219408
Loss: 0.06474382753208976
Loss: 0.06473641066820036
Loss: 0.0647290189618284
Loss: 0.0647216523246021
Loss: 0.064714310

Loss: 0.0631571194561043
Loss: 0.06315500080673664
Loss: 0.06315288909055318
Loss: 0.06315078428436822
Loss: 0.06314868636507602
Loss: 0.0631465953096504
Loss: 0.06314451109514463
Loss: 0.06314243369869094
Loss: 0.06314036309750037
Loss: 0.06313829926886248
Loss: 0.063136242190145
Loss: 0.06313419183879358
Loss: 0.06313214819233164
Loss: 0.0631301112283598
Loss: 0.06312808092455595
Loss: 0.06312605725867466
Loss: 0.0631240402085471
Loss: 0.06312202975208077
Loss: 0.06312002586725907
Loss: 0.06311802853214121
Loss: 0.06311603772486181
Loss: 0.06311405342363072
Loss: 0.06311207560673263
Loss: 0.06311010425252693
Loss: 0.06310813933944746
Loss: 0.06310618084600209
Loss: 0.06310422875077253
Loss: 0.06310228303241416
Loss: 0.06310034366965564
Loss: 0.0630984106412987
Loss: 0.06309648392621783
Loss: 0.06309456350336017
Loss: 0.06309264935174501
Loss: 0.0630907414504638
Loss: 0.06308883977867971
Loss: 0.06308694431562736
Loss: 0.06308505504061271
Loss: 0.0630831719330127
Loss: 0.0630812949722

Loss: 0.06266111133067268
Loss: 0.06266059964152948
Loss: 0.06266008958918494
Loss: 0.06265958116834563
Loss: 0.06265907437373563
Loss: 0.06265856920009624
Loss: 0.06265806564218616
Loss: 0.06265756369478119
Loss: 0.06265706335267443
Loss: 0.06265656461067597
Loss: 0.06265606746361299
Loss: 0.06265557190632974
Loss: 0.06265507793368731
Loss: 0.0626545855405637
Loss: 0.06265409472185379
Loss: 0.0626536054724691
Loss: 0.06265311778733801
Loss: 0.06265263166140546
Loss: 0.062652147089633
Loss: 0.06265166406699878
Loss: 0.06265118258849732
Loss: 0.06265070264913974
Loss: 0.06265022424395338
Loss: 0.062649747367982
Loss: 0.06264927201628558
Loss: 0.0626487981839403
Loss: 0.06264832586603858
Loss: 0.06264785505768883
Loss: 0.06264738575401563
Loss: 0.06264691795015943
Loss: 0.06264645164127675
Loss: 0.06264598682253986
Loss: 0.06264552348913698
Loss: 0.06264506163627209
Loss: 0.06264460125916488
Loss: 0.06264414235305067
Loss: 0.06264368491318052
Loss: 0.06264322893482092
Loss: 0.06264277441

In [32]:
# 試してみましょう！

print (y_true)
y_pred = predict(model, linear_activation, x)
print (y_pred)

[0. 0. 0. 1.]
[-0.26535257  0.24591491  0.24926654  0.76053402]


## 適する活性化関数

### ステップ関数

$$ step(x)=\begin{cases} 1 & x\geq 0.5 \text{ の場合}\\ \\ 0 & \text{以外} \end{cases}$$

<img src="images/step.png" alt="step function" style="width: 650px;"/>

ただ、ステップ関数は微分ができないので、正確に学習できない。それに近い「シグモイド関数」を使おう：

### シグモイド関数

$$ sigmod(x) = \frac{1}{1 + e^{-x}} $$

<img src="images/sigmoid.png" alt="sigmoid function" style="width: 650px;"/>



In [33]:
def sigmoid_activation(x):
    return 1 / (1 + np.power(np.e, -x))

In [34]:
# モデルを再構築し、シグモイド関数の活性化で学習させよ！
np.random.seed(12345)
model = create_perceptron()

In [35]:
fit_epochs(model, sigmoid_activation, mean_squared_error, mse_gradient, 0.01, x, y_true, 5000)

Loss: 0.2553056778192273
Loss: 0.25518538826963366
Loss: 0.25506514190938345
Loss: 0.2549449387449596
Loss: 0.25482477878252324
Loss: 0.25470466202791603
Loss: 0.2545845884866628
Loss: 0.2544645581639743
Loss: 0.25434457106474984
Loss: 0.25422462719357974
Loss: 0.2541047265547483
Loss: 0.25398486915223595
Loss: 0.253865054989722
Loss: 0.2537452840705875
Loss: 0.2536255563979172
Loss: 0.2535058719745026
Loss: 0.2533862308028445
Loss: 0.2532666328851551
Loss: 0.25314707822336097
Loss: 0.2530275668191052
Loss: 0.25290809867375
Loss: 0.25278867378837955
Loss: 0.25266929216380174
Loss: 0.25254995380055134
Loss: 0.25243065869889203
Loss: 0.25231140685881903
Loss: 0.2521921982800613
Loss: 0.2520730329620843
Loss: 0.251953910904092
Loss: 0.25183483210502966
Loss: 0.251715796563586
Loss: 0.25159680427819553
Loss: 0.25147785524704086
Loss: 0.25135894946805537
Loss: 0.25124008693892536
Loss: 0.2511212676570922
Loss: 0.2510024916197549
Loss: 0.25088375882387237
Loss: 0.2507650692661655
Loss: 0.250

Loss: 0.20876895165456485
Loss: 0.20866563726385956
Loss: 0.20856236061717048
Loss: 0.20845912170646214
Loss: 0.2083559205237615
Loss: 0.20825275706115867
Loss: 0.20814963131080633
Loss: 0.2080465432649196
Loss: 0.2079434929157763
Loss: 0.20784048025571658
Loss: 0.20773750527714308
Loss: 0.2076345679725207
Loss: 0.2075316683343762
Loss: 0.2074288063552987
Loss: 0.20732598202793934
Loss: 0.20722319534501082
Loss: 0.207120446299288
Loss: 0.20701773488360714
Loss: 0.20691506109086627
Loss: 0.20681242491402474
Loss: 0.2067098263461034
Loss: 0.20660726538018437
Loss: 0.20650474200941082
Loss: 0.2064022562269871
Loss: 0.2062998080261786
Loss: 0.20619739740031107
Loss: 0.20609502434277155
Loss: 0.20599268884700744
Loss: 0.2058903909065266
Loss: 0.2057881305148974
Loss: 0.20568590766574824
Loss: 0.20558372235276795
Loss: 0.2054815745697052
Loss: 0.20537946431036858
Loss: 0.2052773915686264
Loss: 0.20517535633840683
Loss: 0.20507335861369724
Loss: 0.20497139838854472
Loss: 0.20486947565705538
L

Loss: 0.1456254538577922
Loss: 0.14554854524995933
Loss: 0.14547167484536125
Loss: 0.14539484264202676
Loss: 0.14531804863795938
Loss: 0.14524129283113826
Loss: 0.1451645752195172
Loss: 0.1450878958010252
Loss: 0.14501125457356645
Loss: 0.14493465153502005
Loss: 0.14485808668324018
Loss: 0.14478156001605616
Loss: 0.14470507153127227
Loss: 0.14462862122666775
Loss: 0.14455220909999697
Loss: 0.14447583514898926
Loss: 0.14439949937134908
Loss: 0.14432320176475563
Loss: 0.14424694232686336
Loss: 0.1441707210553015
Loss: 0.14409453794767443
Loss: 0.1440183930015615
Loss: 0.14394228621451685
Loss: 0.1438662175840698
Loss: 0.14379018710772448
Loss: 0.1437141947829601
Loss: 0.14363824060723074
Loss: 0.14356232457796536
Loss: 0.14348644669256808
Loss: 0.1434106069484177
Loss: 0.14333480534286822
Loss: 0.1432590418732483
Loss: 0.1431833165368618
Loss: 0.14310762933098736
Loss: 0.14303198025287844
Loss: 0.14295636929976366
Loss: 0.1428807964688465
Loss: 0.14280526175730524
Loss: 0.142729765162293

Loss: 0.10494150019355283
Loss: 0.10488705574005518
Loss: 0.10483264408177498
Loss: 0.10477826520514869
Loss: 0.10472391909660422
Loss: 0.10466960574256087
Loss: 0.10461532512942923
Loss: 0.1045610772436115
Loss: 0.10450686207150112
Loss: 0.10445267959948343
Loss: 0.10439852981393488
Loss: 0.10434441270122376
Loss: 0.10429032824770995
Loss: 0.10423627643974495
Loss: 0.10418225726367181
Loss: 0.1041282707058254
Loss: 0.10407431675253238
Loss: 0.1040203953901111
Loss: 0.10396650660487165
Loss: 0.10391265038311603
Loss: 0.10385882671113814
Loss: 0.10380503557522366
Loss: 0.10375127696165032
Loss: 0.10369755085668769
Loss: 0.10364385724659735
Loss: 0.10359019611763298
Loss: 0.10353656745604031
Loss: 0.10348297124805705
Loss: 0.10342940747991303
Loss: 0.10337587613783031
Loss: 0.1033223772080231
Loss: 0.10326891067669783
Loss: 0.10321547653005309
Loss: 0.10316207475427977
Loss: 0.10310870533556114
Loss: 0.1030553682600727
Loss: 0.10300206351398242
Loss: 0.10294879108345056
Loss: 0.102895550

Loss: 0.08661484998198833
Loss: 0.08657175570304068
Loss: 0.08652868874653066
Loss: 0.08648564909757536
Loss: 0.08644263674129285
Loss: 0.08639965166280186
Loss: 0.08635669384722185
Loss: 0.08631376327967313
Loss: 0.08627085994527679
Loss: 0.08622798382915484
Loss: 0.08618513491642985
Loss: 0.08614231319222557
Loss: 0.08609951864166654
Loss: 0.08605675124987808
Loss: 0.08601401100198652
Loss: 0.08597129788311914
Loss: 0.08592861187840403
Loss: 0.08588595297297039
Loss: 0.08584332115194832
Loss: 0.08580071640046896
Loss: 0.08575813870366446
Loss: 0.08571558804666791
Loss: 0.08567306441461359
Loss: 0.0856305677926367
Loss: 0.08558809816587368
Loss: 0.08554565551946192
Loss: 0.08550323983854002
Loss: 0.08546085110824761
Loss: 0.08541848931372559
Loss: 0.08537615444011597
Loss: 0.08533384647256184
Loss: 0.0852915653962077
Loss: 0.08524931119619913
Loss: 0.08520708385768291
Loss: 0.08516488336580708
Loss: 0.08512270970572111
Loss: 0.08508056286257548
Loss: 0.0850384428215221
Loss: 0.0849963

...

Loss: 0.0548491996517223
Loss: 0.054826572651309274
Loss: 0.05480396015651588
Loss: 0.054781362157357426
Loss: 0.054758778643854894
Loss: 0.054736209606035605
Loss: 0.05471365503393271
Loss: 0.054691114917585354
Loss: 0.054668589247038823
Loss: 0.05464607801234429
Loss: 0.05462358120355901
Loss: 0.05460109881074619
Loss: 0.05457863082397507
Loss: 0.054556177233320874
Loss: 0.05453373802886488
Loss: 0.054511313200694286
Loss: 0.05448890273890236
Loss: 0.054466506633588305
Loss: 0.054444124874857364
Loss: 0.05442175745282077
Loss: 0.054399404357595674
Loss: 0.054377065579305356
Loss: 0.054354741108078995
Loss: 0.05433243093405173
Loss: 0.05431013504736476
Loss: 0.05428785343816524
Loss: 0.05426558609660631
Loss: 0.05424333301284705
Loss: 0.05422109417705256
Loss: 0.05419886957939398
Loss: 0.0541766592100483
Loss: 0.05415446305919855
Loss: 0.054132281117033776
Loss: 0.054110113373748936
Loss: 0.05408795981954493
Loss: 0.05406582044462872
Loss: 0.05404369523921322
Loss: 0.05402158419351721

Loss: 0.046179653266050794
Loss: 0.046162515831858986
Loss: 0.04614538907257343
Loss: 0.04612827298066324
Loss: 0.0461111675486025
Loss: 0.04609407276887044
Loss: 0.046076988633951335
Loss: 0.046059915136334575
Loss: 0.04604285226851459
Loss: 0.04602580002299084
Loss: 0.04600875839226791
Loss: 0.04599172736885544
Loss: 0.04597470694526809
Loss: 0.045957697114025614
Loss: 0.04594069786765281
Loss: 0.04592370919867954
Loss: 0.045906731099640684
Loss: 0.04588976356307623
Loss: 0.045872806581531075
Loss: 0.04585586014755538
Loss: 0.045838924253704116
Loss: 0.045821998892537436
Loss: 0.04580508405662047
Loss: 0.0457881797385234
Loss: 0.04577128593082143
Loss: 0.04575440262609477
Loss: 0.04573752981692873
Loss: 0.04572066749591358
Loss: 0.04570381565564454
Loss: 0.04568697428872201
Loss: 0.04567014338775127
Loss: 0.04565332294534265
Loss: 0.04563651295411151
Loss: 0.04561971340667826
Loss: 0.04560292429566814
Loss: 0.045586145613711604
Loss: 0.045569377353444004
Loss: 0.04555261950750562
Los

Loss: 0.038053713312945714
Loss: 0.038041512532318425
Loss: 0.038029318929440836
Loss: 0.03801713249922519
Loss: 0.03800495323658743
Loss: 0.03799278113644704
Loss: 0.03798061619372729
Loss: 0.03796845840335507
Loss: 0.03795630776026085
Loss: 0.037944164259378826
Loss: 0.03793202789564684
Loss: 0.037919898664006385
Loss: 0.03790777655940249
Loss: 0.03789566157678397
Loss: 0.03788355371110319
Loss: 0.03787145295731617
Loss: 0.037859359310382576
Loss: 0.03784727276526569
Loss: 0.037835193316932414
Loss: 0.037823120960353285
Loss: 0.03781105569050251
Loss: 0.037798997502357864
Loss: 0.037786946390900675
Loss: 0.03777490235111608
Loss: 0.03776286537799262
Loss: 0.03775083546652259
Loss: 0.03773881261170184
Loss: 0.037726796808529806
Loss: 0.037714788052009604
Loss: 0.03770278633714788
Loss: 0.03769079165895486
Loss: 0.03767880401244449
Loss: 0.03766682339263418
Loss: 0.03765484979454502
Loss: 0.03764288321320164
Loss: 0.03763092364363224
Loss: 0.037618971080868716
Loss: 0.0376070255199464


Loss: 0.03457116151048792
Loss: 0.03456097081097825
Loss: 0.03455078586796947
Loss: 0.034540606677406105
Loss: 0.034530433235235775
Loss: 0.03452026553740886
Loss: 0.034510103579878965
Loss: 0.03449994735860239
Loss: 0.0344897968695386
Loss: 0.03447965210864995
Loss: 0.03446951307190168
Loss: 0.03445937975526214
Loss: 0.03444925215470252
Loss: 0.034439130266197
Loss: 0.03442901408572271
Loss: 0.03441890360925967
Loss: 0.03440879883279098
Loss: 0.03439869975230256
Loss: 0.03438860636378331
Loss: 0.0343785186632251
Loss: 0.0343684366466227
Loss: 0.03435836030997379
Loss: 0.03434828964927909
Loss: 0.03433822466054213
Loss: 0.03432816533976948
Loss: 0.034318111682970526
Loss: 0.034308063686157714
Loss: 0.03429802134534623
Loss: 0.034287984656554395
Loss: 0.034277953615803256
Loss: 0.034267928219116936
Loss: 0.034257908462522364
Loss: 0.03424789434204941
Loss: 0.0342378858537309
Loss: 0.03422788299360252
Loss: 0.03421788575770289
Loss: 0.03420789414207351
Loss: 0.034197908142758845
Loss: 0.

Loss: 0.030606377973203085
Loss: 0.030598358867822337
Loss: 0.030590344000129166
Loss: 0.030582333367188446
Loss: 0.03057432696606723
Loss: 0.030566324793834677
Loss: 0.030558326847562267
Loss: 0.03055033312432351
Loss: 0.030542343621194158
Loss: 0.030534358335252136
Loss: 0.030526377263577473
Loss: 0.030518400403252426
Loss: 0.03051042775136142
Loss: 0.03050245930499098
Loss: 0.03049449506122989
Loss: 0.030486535017168984
Loss: 0.03047857916990133
Loss: 0.030470627516522155
Loss: 0.030462680054128794
Loss: 0.030454736779820763
Loss: 0.030446797690699738
Loss: 0.030438862783869522
Loss: 0.03043093205643605
Loss: 0.030423005505507496
Loss: 0.03041508312819408
Loss: 0.030407164921608214
Loss: 0.030399250882864465
Loss: 0.03039134100907946
Loss: 0.030383435297372065
Loss: 0.030375533744863247
Loss: 0.030367636348676112
Loss: 0.0303597431059359
Loss: 0.03035185401376994
Loss: 0.030343969069307763
Loss: 0.03033608826968099
Loss: 0.03032821161202338
Loss: 0.030320339093470858
Loss: 0.0303124

In [36]:
print(predict(model, sigmoid_activation, x))

[0.00881682 0.08585146 0.23469099 0.76402128]


In [37]:
np.set_printoptions(suppress=True)
print(predict(model, sigmoid_activation, x))

[0.00881682 0.08585146 0.23469099 0.76402128]


## 練習:「OR」を学習させる

In [38]:
np.random.seed(12345)
model = create_perceptron()

In [39]:
y_true = np.array([ 0, 1, 1, 1 ], dtype=float)

In [40]:
fit_epochs(model, sigmoid_activation, mean_squared_error, mse_gradient, 0.01, x, y_true, 5000)

Loss: 0.3735371058243119
Loss: 0.3725351651820836
Loss: 0.3715367514823751
Loss: 0.37054191027329436
Loss: 0.36955068647105216
Loss: 0.3685631243429294
Loss: 0.3675792674907676
Loss: 0.36659915883498956
Loss: 0.3656228405991605
Loss: 0.3646503542950947
Loss: 0.3636817407085172
Loss: 0.36271703988528303
Loss: 0.36175629111816293
Loss: 0.3607995329341976
Loss: 0.3598468030826254
Loss: 0.3588981385233865
Loss: 0.35795357541620704
Loss: 0.35701314911026244
Loss: 0.35607689413442506
Loss: 0.35514484418809306
Loss: 0.3542170321326032
Loss: 0.3532934899832246
Loss: 0.3523742489017342
Loss: 0.3514593391895707
Loss: 0.35054879028156477
Loss: 0.3496426307402431
Loss: 0.34874088825070165
Loss: 0.34784358961604445
Loss: 0.3469507607533838
Loss: 0.34606242669039605
Loss: 0.34517861156242696
Loss: 0.3442993386101413
Loss: 0.34342463017770997
Loss: 0.34255450771152707
Loss: 0.3416889917594498
Loss: 0.3408281019705536
Loss: 0.33997185709539374
Loss: 0.3391202749867655
Loss: 0.33827337260095314
Loss: 0

Loss: 0.22746273223112393
Loss: 0.22729515179641763
Loss: 0.2271275791531227
Loss: 0.2269600097915207
Loss: 0.2267924392570707
Loss: 0.22662486314993688
Loss: 0.22645727712452063
Loss: 0.22628967688899831
Loss: 0.22612205820486317
Loss: 0.22595441688647286
Loss: 0.2257867488006015
Loss: 0.22561904986599673
Loss: 0.22545131605294116
Loss: 0.2252835433828193
Loss: 0.22511572792768866
Loss: 0.22494786580985537
Loss: 0.22477995320145505
Loss: 0.22461198632403775
Loss: 0.22444396144815762
Loss: 0.22427587489296705
Loss: 0.2241077230258153
Loss: 0.2239395022618516
Loss: 0.2237712090636324
Loss: 0.22360283994073313
Loss: 0.22343439144936428
Loss: 0.22326586019199166
Loss: 0.2230972428169608
Loss: 0.22292853601812565
Loss: 0.22275973653448106
Loss: 0.22259084114980004
Loss: 0.2224218466922743
Loss: 0.22225275003415892
Loss: 0.22208354809142158
Loss: 0.22191423782339478
Loss: 0.22174481623243253
Loss: 0.22157528036357085
Loss: 0.22140562730419167
Loss: 0.2212358541836908
Loss: 0.221065958173149

Loss: 0.13932687298704663
Loss: 0.1391266583280811
Loss: 0.138926667659121
Loss: 0.1387269022243284
Loss: 0.1385273632592167
Loss: 0.13832805199063206
Loss: 0.1381289696367351
Loss: 0.13793011740698344
Loss: 0.13773149650211508
Loss: 0.13753310811413208
Loss: 0.137334953426285
Loss: 0.13713703361305812
Loss: 0.1369393498401555
Loss: 0.13674190326448693
Loss: 0.13654469503415553
Loss: 0.13634772628844494
Loss: 0.13615099815780823
Loss: 0.13595451176385637
Loss: 0.13575826821934828
Loss: 0.13556226862818066
Loss: 0.1353665140853792
Loss: 0.1351710056770898
Loss: 0.1349757444805711
Loss: 0.13478073156418638
Loss: 0.13458596798739758
Loss: 0.13439145480075906
Loss: 0.13419719304591193
Loss: 0.13400318375557924
Loss: 0.13380942795356177
Loss: 0.13361592665473424
Loss: 0.1334226808650422
Loss: 0.13322969158149953
Loss: 0.13303695979218633
Loss: 0.1328444864762478
Loss: 0.13265227260389326
Loss: 0.13246031913639575
Loss: 0.1322686270260927
Loss: 0.13207719721638644
Loss: 0.131886030641746
Los

Loss: 0.07065983685388448
Loss: 0.07059881383279996
Loss: 0.07053793027237228
Loss: 0.07047718575415539
Loss: 0.07041657986074047
Loss: 0.07035611217575627
Loss: 0.07029578228386937
Loss: 0.07023558977078397
Loss: 0.07017553422324256
Loss: 0.07011561522902562
Loss: 0.07005583237695179
Loss: 0.06999618525687804
Loss: 0.06993667345969944
Loss: 0.06987729657734934
Loss: 0.06981805420279921
Loss: 0.06975894593005866
Loss: 0.06969997135417519
Loss: 0.06964113007123414
Loss: 0.06958242167835858
Loss: 0.06952384577370889
Loss: 0.06946540195648285
Loss: 0.06940708982691518
Loss: 0.06934890898627737
Loss: 0.06929085903687715
Loss: 0.0692329395820585
Loss: 0.0691751502262012
Loss: 0.06911749057472015
Loss: 0.06905996023406538
Loss: 0.06900255881172138
Loss: 0.06894528591620663
Loss: 0.06888814115707333
Loss: 0.06883112414490661
Loss: 0.06877423449132421
Loss: 0.06871747180897585
Loss: 0.06866083571154265
Loss: 0.06860432581373647
Loss: 0.06854794173129951
Loss: 0.06849168308100333
Loss: 0.068435

Loss: 0.04852159463927199
Loss: 0.0484994976268961
Loss: 0.048477423812975165
Loss: 0.04845537314071567
Loss: 0.0484333455535147
Loss: 0.048411340994959036
Loss: 0.048389359408824585
Loss: 0.04836740073907571
Loss: 0.04834546492986449
Loss: 0.04832355192553014
Loss: 0.048301661670598274
Loss: 0.048279794109780345
Loss: 0.048257949187972805
Loss: 0.048236126850256654
Loss: 0.048214327041896654
Loss: 0.048192549708340686
Loss: 0.04817079479521917
Loss: 0.048149062248344345
Loss: 0.048127352013709594
Loss: 0.04810566403748898
Loss: 0.048083998266036376
Loss: 0.048062354645884944
Loss: 0.048040733123746474
Loss: 0.04801913364651082
Loss: 0.04799755616124512
Loss: 0.04797600061519328
Loss: 0.04795446695577537
Loss: 0.04793295513058682
Loss: 0.04791146508739804
Loss: 0.04788999677415358
Loss: 0.04786855013897167
Loss: 0.04784712513014357
Loss: 0.04782572169613278
Loss: 0.04780433978557469
Loss: 0.04778297934727584
Loss: 0.04776164033021331
Loss: 0.047740322683534106
Loss: 0.04771902635655456

...

Loss: 0.030858675150099348
Loss: 0.030848291556204967
Loss: 0.030837912792530727
Loss: 0.03082753885601994
Loss: 0.03081716974361969
Loss: 0.030806805452280938
Loss: 0.03079644597895841
Loss: 0.030786091320610613
Loss: 0.03077574147419983
Loss: 0.030765396436692138
Loss: 0.030755056205057363
Loss: 0.030744720776269076
Loss: 0.03073439014730457
Loss: 0.030724064315144888
Loss: 0.03071374327677483
Loss: 0.030703427029182825
Loss: 0.030693115569361048
Loss: 0.030682808894305396
Loss: 0.03067250700101539
Loss: 0.03066220988649427
Loss: 0.030651917547748878
Loss: 0.03064162998178982
Loss: 0.030631347185631225
Loss: 0.030621069156290946
Loss: 0.03061079589079039
Loss: 0.03060052738615466
Loss: 0.030590263639412398
Loss: 0.03058000464759589
Loss: 0.030569750407740987
Loss: 0.03055950091688714
Loss: 0.03054925617207732
Loss: 0.030539016170358143
Loss: 0.030528780908779713
Loss: 0.03051855038439571
Loss: 0.03050832459426334
Loss: 0.03049810353544333
Loss: 0.030487887204999915
Loss: 0.0304776756

Loss: 0.026794882568126167
Loss: 0.026786337297275618
Loss: 0.02677779579757773
Loss: 0.02676925806701726
Loss: 0.026760724103580467
Loss: 0.026752193905255135
Loss: 0.026743667470030565
Loss: 0.026735144795897545
Loss: 0.026726625880848405
Loss: 0.026718110722876973
Loss: 0.02670959931997859
Loss: 0.026701091670150095
Loss: 0.026692587771389803
Loss: 0.0266840876216976
Loss: 0.026675591219074753
Loss: 0.026667098561524123
Loss: 0.02665860964705004
Loss: 0.026650124473658277
Loss: 0.02664164303935615
Loss: 0.02663316534215243
Loss: 0.026624691380057373
Loss: 0.026616221151082718
Loss: 0.02660775465324166
Loss: 0.026599291884548893
Loss: 0.026590832843020596
Loss: 0.02658237752667438
Loss: 0.026573925933529318
Loss: 0.026565478061606026
Loss: 0.026557033908926476
Loss: 0.02654859347351418
Loss: 0.02654015675339406
Loss: 0.0265317237465925
Loss: 0.026523294451137394
Loss: 0.02651486886505798
Loss: 0.02650644698638507
Loss: 0.026498028813150803
Loss: 0.02648961434338885
Loss: 0.0264812035

Loss: 0.02422027300472794
Loss: 0.024212848537695772
Loss: 0.02420542725868783
Loss: 0.02419800916608156
Loss: 0.024190594258255406
Loss: 0.024183182533588782
Loss: 0.024175773990462095
Loss: 0.024168368627256735
Loss: 0.02416096644235511
Loss: 0.024153567434140526
Loss: 0.024146171600997357
Loss: 0.024138778941310907
Loss: 0.024131389453467482
Loss: 0.02412400313585434
Loss: 0.02411661998685974
Loss: 0.02410924000487291
Loss: 0.024101863188284063
Loss: 0.02409448953548434
Loss: 0.02408711904486592
Loss: 0.024079751714821913
Loss: 0.024072387543746396
Loss: 0.024065026530034454
Loss: 0.02405766867208207
Loss: 0.024050313968286285
Loss: 0.024042962417045025
Loss: 0.02403561401675726
Loss: 0.02402826876582285
Loss: 0.02402092666264265
Loss: 0.024013587705618473
Loss: 0.024006251893153117
Loss: 0.023998919223650314
Loss: 0.023991589695514758
Loss: 0.02398426330715213
Loss: 0.02397694005696901
Loss: 0.023969619943372978
Loss: 0.023962302964772603
Loss: 0.023954989119577316
Loss: 0.02394767

Loss: 0.020856424211087837
Loss: 0.020850417041162903
Loss: 0.020844412350642112
Loss: 0.020838410138304397
Loss: 0.020832410402929335
Loss: 0.02082641314329715
Loss: 0.020820418358188753
Loss: 0.020814426046385698
Loss: 0.0208084362066702
Loss: 0.020802448837825106
Loss: 0.020796463938633974
Loss: 0.02079048150788095
Loss: 0.02078450154435092
Loss: 0.020778524046829346
Loss: 0.02077254901410238
Loss: 0.020766576444956823
Loss: 0.020760606338180148
Loss: 0.02075463869256043
Loss: 0.02074867350688649
Loss: 0.02074271077994769
Loss: 0.020736750510534134
Loss: 0.020730792697436533
Loss: 0.020724837339446252
Loss: 0.020718884435355325
Loss: 0.020712933983956426
Loss: 0.020706985984042865
Loss: 0.020701040434408666
Loss: 0.020695097333848392
Loss: 0.020689156681157363
Loss: 0.020683218475131476
Loss: 0.02067728271456732
Loss: 0.02067134939826211
Loss: 0.020665418525013697
Loss: 0.020659490093620614
Loss: 0.02065356410288202
Loss: 0.02064764055159772
Loss: 0.02064171943856818
Loss: 0.0206358

Loss: 0.018522215581726436
Loss: 0.018517155968944852
Loss: 0.018512098373201603
Loss: 0.018507042793522162
Loss: 0.018501989228932522
Loss: 0.01849693767845918
Loss: 0.018491888141129183
Loss: 0.01848684061597006
Loss: 0.018481795102009847
Loss: 0.018476751598277146
Loss: 0.018471710103801036
Loss: 0.018466670617611113
Loss: 0.01846163313873746
Loss: 0.018456597666210734
Loss: 0.018451564199062062
Loss: 0.018446532736323077
Loss: 0.01844150327702597
Loss: 0.01843647582020339
Loss: 0.018431450364888514
Loss: 0.018426426910115062
Loss: 0.018421405454917234
Loss: 0.018416385998329746
Loss: 0.01841136853938781
Loss: 0.01840635307712719
Loss: 0.01840133961058412
Loss: 0.018396328138795364
Loss: 0.018391318660798184
Loss: 0.018386311175630363
Loss: 0.01838130568233019
Loss: 0.018376302179936445
Loss: 0.018371300667488447
Loss: 0.018366301144025998
Loss: 0.01836130360858941
Loss: 0.018356308060219536
Loss: 0.018351314497957676
Loss: 0.018346322920845676
Loss: 0.018341333327925893
Loss: 0.018

In [41]:
print(predict(model, sigmoid_activation, x))

[0.22701713 0.86221884 0.97794843 0.99894288]


In [42]:
print(model)

{'weights': array([5.01730468, 3.05907396]), 'bias': array([-1.22523144])}
